In [45]:
import pandas as pd
import json
import requests
import numpy as np

In [46]:
def fetch_data_from_url(url):
    """
    Fetch JSON data from the given URL.
    """
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad responses
    return response.json()

def normalize_restaurant_data(restaurant_data):
    """
    Normalize the "restaurants" object in JSON data to a DataFrame.
    """
    return pd.json_normalize(restaurant_data, 'restaurants')

def merge_with_country_code(restaurant_df, country_code_df):
    """
    Merge restaurant data with country code DataFrame based on country ID.
    """
    return pd.merge(restaurant_df, country_code_df, 
                    how='left', left_on='restaurant.location.country_id', 
                    right_on='Country Code')

def extract_required_fields(merged_df):
    """
    Extract required fields from the merged DataFrame.
    """
    return merged_df[['restaurant.R.res_id', 'restaurant.name', 'Country', 
                      'restaurant.location.city', 'restaurant.user_rating.votes', 
                      'restaurant.user_rating.aggregate_rating', 'restaurant.cuisines']]

def rename_columns(final_df):
    """
    Rename columns of the DataFrame.
    """
    renamed_columns = {
        'restaurant.R.res_id': 'Restaurant Id',
        'restaurant.name': 'Restaurant Name',
        'restaurant.location.city': 'City',
        'restaurant.user_rating.votes': 'User Rating Votes',
        'restaurant.user_rating.aggregate_rating': 'User Aggregate Rating',
        'restaurant.cuisines': 'Cuisines'
    }
    return final_df.rename(columns=renamed_columns)

def convert_data_types(final_df):
    """
    Convert data types of specific columns in the DataFrame.
    """
    final_df['User Aggregate Rating'] = final_df['User Aggregate Rating'].astype('float64')
    return final_df

def export_to_csv(final_df, file_path):
    """
    Export DataFrame to a CSV file.
    """
    final_df.to_csv(file_path, index=False)

In [56]:
# URL of the JSON data
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

# Fetch data from the URL
restaurant_data = fetch_data_from_url(url)

# Normalize restaurant data
restaurant_df = normalize_restaurant_data(restaurant_data)

In [ ]:
restaurant_df

In [ ]:
# Unpack the lists in the events column vertically
expended_events_df = restaurant_df.explode("restaurant.zomato_events")

# Unpack the dictionaries within each element of the column
unpacked_events_df = pd.json_normalize(expended_events_df["restaurant.zomato_events"])

# Merge the unpacked DataFrame with the original DataFrame
events_df = pd.concat([restaurant_df, unpacked_events_df], axis=1)

In [ ]:
restaurant_df['restaurant.zomato_events'][1117]

In [ ]:
def main():
    # URL of the JSON data
    url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
    
    # Fetch data from the URL
    restaurant_data = fetch_data_from_url(url)
    
    # Normalize restaurant data
    restaurant_df = normalize_restaurant_data(restaurant_data)

    # Read in Country-Code Excel file into DataFrame
    country_code_df = pd.read_csv("Country-Code.csv")
    
    # Merge restaurant data with country code data
    merged_restaurant_df = merge_with_country_code(restaurant_df, country_code_df)
    
    # Extract required fields
    final_restaurant_df = extract_required_fields(merged_restaurant_df)
    
    # Rename columns
    final_restaurant_df = rename_columns(final_restaurant_df)
    
    # Convert data types
    final_restaurant_df = convert_data_types(final_restaurant_df)
    
    # Input file path to export restaurants.csv file
    export_to_csv(final_restaurant_df, "C:\\Users\\junke\\Desktop\\Important Documents (JunKeat)\\GovTech-Application\\restaurants.csv")